## PID Command, Control and Communication

This code can be used to send commands to the PR59 PID temperature controller.
It shows the in real-time temperature of the thermostate, the interactions enable the user to send commands directly to the PID controller.

In [24]:
import serial
import time
import numpy as np

# to remove werid behavior where data polling gets slower when plot window is not in focus
# https://stackoverflow.com/questions/1023038/change-process-priority-in-python-cross-platform
import psutil, os, sys
#p.nice(psutil.HIGH_PRIORITY_CLASS)
# os.nice(-10) # for linux/macOS

p = psutil.Process(os.getpid())
os_used = sys.platform
if os_used == "win32":  # Windows (either 32-bit or 64-bit)
    p.nice(psutil.REALTIME_PRIORITY_CLASS)
elif os_used == "linux":  # linux
    p.nice(psutil.IOPRIO_HIGH)
else:  # MAC OS X or other
    p.nice(-20)  


# Connections set up through USB-serial converter
ser = serial.Serial()
ser.port = "COM7"
ser.baudrate = 115200   
ser.timeout = 1
ser.rts = False
ser.dtr = False
ser.open()

# Main function, sending and receiving data to the device
def query(message, flagR=True, flagW=True):
    if flagW:    
        message += '\n\r'
        ser.write(message.encode('utf-8'))
        res = ser.readline().decode()
    if flagR:
        res = ser.readline().decode()
        return res
    
# testing the connection
print(query("$V?"))
time.sleep(1)



SC_v2.9



In [26]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 

flag_testing = False
temp = []


# -[Funzioni di gestione eventi]-----------------------------------------------
def on_close(event):
    global flag_run
    flag_run = False
def on_key(event):
    global flag_run
    global flag_acq

    if event.key == 'x':  # ⇒ export su file
        filename = input("Esporta dati su file: ")
        dataposition = '../../../Data/'
        data = np.column_stack((np.arange(0, len(temp)), temp))
        info = "# Data acquired from temp PID controller on "+time.strftime("%Y-%m-%d %H:%M:%S")+"\n"
        info += "# Data format: [Sample, Temperature [°C]]\n"
        np.savetxt(dataposition+filename+'.txt', data, delimiter='\t', header=info)

    if event.key == 'c':  # ⇒ send command to scope
        message = input(">> ")
        print(query(message))

    if event.key == ' ':  # ⇒ run/pausa misura 
        flag_acq = not flag_acq

    if event.key == 'escape':  # ⇒ esci dalla misura
        flag_run = False


###    HIGHLY DANGEROUS - DO NOT BE A SILLY GOOSE
    if event.key == 'e':  # ⇒ erase all previous data
        temp.clear()

        print("Erased all data!")
        print("You are a silly goose!")

    

# -[Ciclo di misura]-----------------------------------------------------------
fig, axs = plt.subplots(1,2,figsize=(12,6))
fig.canvas.mpl_connect("close_event", on_close)
fig.canvas.mpl_connect('key_press_event', on_key)
flag_run = True
flag_acq = True
flag_first = True


while flag_run:
    if flag_acq: # l'acquisizione è attiva?
        if flag_testing:
            data = np.sin(len(temp)/10) * .1 + np.sin(len(temp)/2) * .05 # test data with obvious pattern
        else:
            data = float(query("$R100?"))

        temp.append(data)

    # Visualizzazione
    if flag_first:
        flag_first = False
        plt.show(block = False)

    else:
        axs[0].clear()
        axs[1].clear()

        fig.suptitle("User interaction: x|c|space|escape|e(DONT BE SILLY, GOOSE)")

        axs[0].set_title("Full data acquisition")
        axs[0].plot(np.arange(0, len(temp)), temp, "-", label="Temp Data", color="tab:blue")
        axs[0].set_xlabel("Sample", fontsize=15)
        axs[0].set_ylabel("Temperature [°C]", fontsize=15)
        axs[0].legend(loc='upper right')
        axs[0].grid(linestyle='-.')

        secondGraphMax = 200
        if(len(temp) < 200):
            secondGraphMax = len(temp)

        axs[1].set_title("Latest 200 samples")
        axs[1].plot(np.arange(len(temp)-secondGraphMax, len(temp)), temp[-secondGraphMax:], "-", label="Latest Temp Data", color="tab:orange")
        axs[1].set_xlabel("Sample", fontsize=15)
        axs[1].set_ylabel("Temperature [°C]", fontsize=15)
        axs[1].annotate(f"Latest Value [°C]: {temp[-1]:.2f}", (0.4,0.96), xycoords='axes fraction', fontsize=12)
        axs[1].legend(loc='upper right')
        axs[1].grid(linestyle='-.')
#        axs[1].set_yticks(np.arange(min(temp[-200:]), max(temp[-200:]), 20))

        print(temp[-1])
        fig.canvas.draw()
        fig.canvas.flush_events()









8.458229
8.489837
8.506881
8.518783
8.538658
8.556945
8.567909
8.578895
8.597221
8.609436
8.630195
8.64241
8.660766
8.673004
8.688919
8.706077
8.717102
8.733032
8.7453
8.761238
8.775978
8.788246
8.805458
8.81652
8.833732
8.844802
8.860801
8.871879


In [27]:
ser.close()